In [ ]:
import pandas as pd

In [ ]:
from pandas import Series, DataFrame

In [ ]:
from __future__ import print_function

In [ ]:
import xlrd, psycopg2, os, xlsxwriter

In [ ]:
conn = psycopg2.connect(dbname="nvdpsarrests", user="acohen", password=os.environ['PGPASSWORD'], host = os.environ['AWS_PG_HOST'])

In [ ]:
cur = conn.cursor()

In [ ]:
datadir = '/Users/acohen/data/NV_DPS_Arrests/data/'

In [ ]:
excelfile = datadir + 'criminal_matches.xlsx'

In [ ]:
handle = pd.ExcelFile(excelfile)

In [ ]:
matches = handle.parse(handle.sheet_names[2])

In [ ]:
matches.head()

In [ ]:
query = """
SELECT DISTINCT t1.*,t4.description,t2.*
FROM arrests.arrest t1
JOIN arrests.offense t2 ON t1.arrest_rec_num = t2.arrest_rec_num
JOIN arrests.arrest t3 ON t1.bin = t3.bin
LEFT JOIN arrests.offenseclasscode t4 ON TRIM(t2.offense_class) = t4.code
WHERE t3.last_name = %s AND t3.first_name = %s
"""

In [ ]:
frames = []
for row in matches[['last_name','first_name']].itertuples():
        frame = pd.read_sql_query(query, conn,params=[row[1],row[2]])
        frames.append(frame)

In [ ]:
result = pd.concat(frames, ignore_index=True)

In [ ]:
excel_date = str(pd.to_datetime('today')).split(' ')[0].replace('-','')

In [ ]:
excel_name = datadir + 'immigrant_felons' + excel_date + '.xlsx'

In [ ]:
writer = pd.ExcelWriter(excel_name, engine='xlsxwriter')

In [ ]:
result.to_excel(writer,sheet_name='Sheet1')

In [ ]:
writer.save()

In [ ]:
!open $excel_name